# Performance Assessement

D206 Data Cleaning </br>
David Rusho </br>
Student ID: 012037553 </br>

---


## Import the Data


#### Install Python Libraries


In [222]:
# !pip3 install matplotlib
# !pip3 install pandas
# !pip3 install prettytable
# !pip3 install scipy
# !pip3 install missingno

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 6.7 MB/s eta 0:00:00a 0:00:01


#### Import Libaries


In [223]:
# Import Libraries
import numpy as np
from matplotlib import pyplot as plt
import missingno as msno
import pandas as pd
import prettytable
from scipy import stats

#### Load CSV into Pandas


In [201]:
# Increase column display/print range to display all columns
pd.set_option("display.max_columns", None)

# Specify the file path
file_path = "/Users/david/Documents/GitHub/WGU/d206 - Data Cleaning/Assessment/Medical Data and Dictionary Files/medical_raw_data.csv"

# Read the CSV file into a DataFrame
# First column of csv is already an index, set Pandas to not create an index  using 'index_col=0'
med_df = pd.read_csv(file_path, index_col=0)

# print first 3 rows
med_df.head(3)

,CaseOrder,Customer_id,Interaction,UID,City,State,County,Zip,Lat,Lng,Population,Area,Timezone,Job,Children,Age,Education,Employment,Income,Marital,Gender,ReAdmis,VitD_levels,Doc_visits,Full_meals_eaten,VitD_supp,Soft_drink,Initial_admin,HighBlood,Stroke,Complication_risk,Overweight,Arthritis,Diabetes,Hyperlipidemia,BackPain,Anxiety,Allergic_rhinitis,Reflux_esophagitis,Asthma,Services,Initial_days,TotalCharge,Additional_charges,Item1,Item2,Item3,Item4,Item5,Item6,Item7,Item8
1,1,C412403,8cd49b13-f45a-4b47-a2bd-173ffa932c2f,3a83ddb66e2ae73798bdf1d705dc0932,Eva,AL,Morgan,35621,34.34960,-86.72508,2951,Suburban,America/Chicago,"Psychologist, sport and exercise",1.0,53.0,"Some College, Less than 1 Year",Full Time,86575.93,Divorced,Male,No,17.802330,6,0,0,NaN,Emergency Admission,Yes,No,Medium,0.0,Yes,Yes,No,Yes,1.0,Yes,No,Yes,Blood Work,10.585770,3191.048774,17939.40342,3,3,2,2,4,3,3,4
2,2,Z919181,d2450b70-0337-4406-bdbb-bc1037f1734c,176354c5eef714957d486009feabf195,Marianna,FL,Jackson,32446,30.84513,-85.22907,11303,Urban,America/Chicago,Community development worker,3.0,51.0,"Some College, 1 or More Years, No Degree",Full Time,46805.99,Married,Female,No,18.994640,4,2,1,No,Emergency Admission,Yes,No,High,1.0,No,No,No,No,NaN,No,Yes,No,Intravenous,15.129562,4214.905346,17612.99812,3,4,3,4,4,4,3,3
3,3,F995323,a2057123-abf5-4a2c-abad-8ffe33512562,e19a0fa00aeda885b8a436757e889bc9,Sioux Falls,SD,Minnehaha,57110,43.54321,-96.63772,17125,Suburban,America/Chicago,Chief Executive Officer,3.0,53.0,"Some College, 1 or More Years, No Degree",Retired,14370.14,Widowed,Female,No,17.415889,4,1,0,No,Elective Admission,Yes,No,Medium,1.0,No,Yes,No,No,NaN,No,No,No,Blood Work,4.772177,2177.586768,17505.19246,2,4,4,4,3,4,3,3


### Data Profiling


In [199]:
# Display dataframe datatypes and other basic info
med_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10000 entries, 1 to 10000
Data columns (total 52 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   CaseOrder           10000 non-null  int64  
 1   Customer_id         10000 non-null  object 
 2   Interaction         10000 non-null  object 
 3   UID                 10000 non-null  object 
 4   City                10000 non-null  object 
 5   State               10000 non-null  object 
 6   County              10000 non-null  object 
 7   Zip                 10000 non-null  int64  
 8   Lat                 10000 non-null  float64
 9   Lng                 10000 non-null  float64
 10  Population          10000 non-null  int64  
 11  Area                10000 non-null  object 
 12  Timezone            10000 non-null  object 
 13  Job                 10000 non-null  object 
 14  Children            7412 non-null   float64
 15  Age                 7586 non-null   float64
 16  Education

### Function to Print Value Counts


In [206]:
# function to print value counts
def print_counts(data_counts, column_name, max_rows=None):
    sorted_counts = data_counts.sort_values(ascending=False)
    table = prettytable.PrettyTable([column_name, "Count"])  # Create table with headers
    if max_rows:
        sorted_counts = sorted_counts.head(max_rows)
    for value, count in sorted_counts.items():
        table.add_row([value, count])
    return table

---

## Data Cleaning


### Detecting and Treat Duplicates


#### Detect Duplicates


In [213]:
# Determine duplicate count of dataframe
# Resource: https://note.nkmk.me/en/python-pandas-duplicated-drop-duplicates/
dup_count = med_df.duplicated().value_counts()

# value counts of duplicates in descending order
print("Duplicate Count")
print_counts(dup_count, "duplicates")

Duplicate Count


duplicates,Count
False,10000


#### Drop Duplicates


In [212]:
# Create a copy of the original dataframe
df_dup_drop = med_df.copy()

# Determine duplicate count of new dataframe copy
# Resource: https://note.nkmk.me/en/python-pandas-duplicated-drop-duplicates/
df_dup_drop = med_df.drop_duplicates()
dup_count2 = df_dup_drop.duplicated().value_counts()

# value counts of duplicates in descending order
print("Duplicate Count After Duplicates Dropped")
print_counts(dup_count2, "duplicates")

Duplicate Count After Duplicates Dropped


duplicates,Count
False,10000


### Detection and Teatment of Missing Values

_Missing values are usually represented in a form of Nan or null or None in the dataset_


#### Count Nulls


In [221]:
# count nulls by column
df_isnull = df_dup_drop.isnull().sum(axis=0)

# value counts of nulls in descending order
print("Top 10 Null Counts by Column Name")
print_counts(df_isnull, "Name", 10)  # .plot(kind="barh")

Top 10 Null Counts by Column Name


Name,Count
Children,2588
Soft_drink,2467
Income,2464
Age,2414
Initial_days,1056
Anxiety,984
Overweight,982
Complication_risk,0
Arthritis,0
Diabetes,0


## Initial Analysis of the Data


### Verify Unique Value Counts

**Columns that should have unqie values per row**

| name        | requied to be unique | description                                                            |
| :---------- | :------------------- | :--------------------------------------------------------------------- |
| CaseOrder   | Yes                  | Original index for dataframe                                           |
| Customer_id | Yes                  | Unique patient ID                                                      |
| Interaction | Yes                  | Unique IDs related to patient transactions, procedures, and admissions |
| UID         | Yes                  | Unique IDs related to patient transactions, procedures, and admissions |

</br>

**Columns that have a high likihood of returning unique values per row**</br>
_Not a requirement of the data cleaning process_
| name | requied to be unique | description |
|:-- |:-- | :-- |
|VitD_levels| No | Patient’s vitamin D levels as measured in ng/mL. |
|TotalCharge| No | The amount charged to the patient daily. |


In [216]:
# Verify unique values count by all columns

# dataframe columns to list
df_cols_list = med_df.columns.to_list()

# count unique values per column,
unique_count = med_df[df_cols_list].nunique()

# print table, limit results to top 10
create_pt(unique_count, "Name", 15)

Name,Count
CaseOrder,10000
VitD_levels,10000
Customer_id,10000
TotalCharge,10000
Interaction,10000
UID,10000
Initial_days,8944
Additional_charges,8888
Zip,8612
Lng,8601


### Verify categorical columns values

_These columns should return values that are limited in count, and are not unique per row_

- Area
- Timezone
- Education
- Employment
- Marital
- Gender
- Initial_admin
- Complication_risk
- Services
- Item1
- Item2
- Item3
- Item4
- Item5
- Item6
- Item7
- Item8


In [197]:
# Verify value counts for all three categorical columns
for col in [
    "Area",
    "Timezone",
    "Education",
    "Employment",
    "Marital",
    "Gender",
    "Initial_admin",
    "Complication_risk",
    "Services",
    "Item1",
    "Item2",
    "Item3",
    "Item4",
    "Item5",
    "Item6",
    "Item7",
    "Item8",
]:
    value_counts = med_df[col].value_counts()

    print(f"Value Counts for '{col}':")
    # print(value_counts)
    print(create_pt(value_counts, "Name"))
    print()

Value Counts for 'Area':
+----------+-------+
|   Name   | Count |
+----------+-------+
|  Rural   |  3369 |
| Suburban |  3328 |
|  Urban   |  3303 |
+----------+-------+

Value Counts for 'Timezone':
+--------------------------------+-------+
|              Name              | Count |
+--------------------------------+-------+
|        America/New_York        |  3889 |
|        America/Chicago         |  3771 |
|      America/Los_Angeles       |  937  |
|         America/Denver         |  612  |
|        America/Detroit         |  262  |
|  America/Indiana/Indianapolis  |  151  |
|        America/Phoenix         |  100  |
|         America/Boise          |   86  |
|       America/Anchorage        |   50  |
|      America/Puerto_Rico       |   43  |
|        Pacific/Honolulu        |   34  |
|       America/Menominee        |   14  |
|          America/Nome          |   12  |
|   America/Indiana/Vincennes    |   8   |
|  America/Kentucky/Louisville   |   6   |
|         America/Sitka 

In [188]:
# print value counts for Area column
area_counts = med_df["Area"].value_counts()
create_pt(area_counts, "Area")

Area,Count
Rural,3369
Suburban,3328
Urban,3303


In [189]:
# print value counts for Timezone column
tz_counts = med_df["Timezone"].value_counts()
create_pt(tz_counts, "Timezone")

Timezone,Count
America/New_York,3889
America/Chicago,3771
America/Los_Angeles,937
America/Denver,612
America/Detroit,262
America/Indiana/Indianapolis,151
America/Phoenix,100
America/Boise,86
America/Anchorage,50
America/Puerto_Rico,43


### Verify datatype Int64 columns

- Children
- Age
-
